In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary

from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

10504699904
0
0


In [2]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [4]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [5]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(.7, .2, .1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  
    

In [6]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original
        else:
            return img, target
    
    def __len__(self):
        return len(self.imgs_files)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as functional


class GNNEncoder(nn.Module):
    def __init__(self, vertices_dim=3, hidden_dim=128, num_vertices=6):
        super(GNNEncoder, self).__init__()
        # Define your layers here, e.g., 
        self.f_enc = nn.Linear(vertices_dim, hidden_dim)  # Assuming 3 features per vertex as per the paper
        self.f_e1 = nn.Linear(hidden_dim * 2, hidden_dim)  # Concatenate two vertices features
        self.f_v = nn.Linear(hidden_dim, hidden_dim)  # Hidden layer for vertices
        self.f_e2 = nn.Linear(hidden_dim * 2, hidden_dim)  # Concatenate updated vertices features

        # Define the graph edges
        self.edges = self.generate_edges(num_vertices).to(device)
                
    def generate_edges(self, num_vertices):
        edges = torch.tensor([[i, i+1] for i in range(num_vertices-1)], dtype=torch.long)
        return edges

    def forward(self, vertices):
        h1 = self.f_enc(vertices)
        
        # using predefined edges from predicted keypoints
        edges = self.edges
        
#         print("keypoints shape", vertices.shape)
#         print("edges.shape: ", edges.shape)        
#         print("max edge index: ", torch.max(edges).item())
        print("edges[:,0]: ", edges[:,0])
        print("edges[:,1]: ", edges[:,1])
        print("h1 that is problematic", h1.shape)
        if h1.shape[0] < num_vertices:  # Adjusting for instances where h1's shape is (5,128) instead of (6,128)
            h1_source = h1[edges[:-1, 0]]  # Only take until the second to last element
            h1_target = h1[edges[:-1, 1]]
            h_e1 = self.f_e1(torch.cat((h1_source, h1_target), dim=1))  # Concatenate along the last dimension
            h2 = torch.tanh(h1)
            h2_prob = torch.sigmoid(h2)  # vertices probability
            h2_source = h2[edges[:-1, 0]]
            h2_target = h2[edges[:-1, 1]]
            h_e2 = self.f_e2(torch.cat((h2_source, h2_target), dim=1))  # v->e
            return h2_prob, h_e2, edges
        else:
            h1_source = h1[edges[:, 0]]
            h1_target = h1[edges[:, 1]]
            h_e1 = self.f_e1(torch.cat((h1_source, h1_target), dim=1))  # Concatenate along the last dimension
            h2 = torch.tanh(h1)
            h2_prob = torch.sigmoid(h2)  # vertices probability
            h2_source = h2[edges[:, 0]]
            h2_target = h2[edges[:, 1]]
            h_e2 = self.f_e2(torch.cat((h2_source, h2_target), dim=1))  # v->e
            return h2_prob, h_e2, edges

class GNNDecoder(nn.Module):
    def __init__(self, hidden_dim=128, vertices_dim=3, num_vertices=6):
        super(GNNDecoder, self).__init__()
        # Define your layers here
        self.f_ep = nn.Linear(2 * hidden_dim, hidden_dim)
        self.f_v = nn.Linear(hidden_dim, hidden_dim)  
        self.f_mu = nn.Linear(hidden_dim, vertices_dim)        

    def forward(self, enc_vertices, enc_edges, edges):
        
        if enc_vertices.shape[0] < num_vertices:  # Adjusting for instances where enc_vertices's shape is less than expected
            # Get the embeddings of the source and target nodes
            vertices_source = enc_vertices[edges[:-1, 0]]
            vertices_target = enc_vertices[edges[:-1, 1]]
            # compute v -> e message
            h_e = self.f_ep(torch.cat((vertices_source, vertices_target), dim=1))  # v->e
            h_e = torch.tanh(h_e)
            # compute e -> v message
            h_v = torch.zeros_like(enc_vertices)
            h_v = h_v.scatter_add(0, edges[:-1, 1].unsqueeze(1).repeat(1, enc_vertices.size(1)), h_e)
            
        else:
            vertices_source = enc_vertices[edges[:, 0]]
            vertices_target = enc_vertices[edges[:, 1]]
            # compute v -> e message
            h_e = self.f_ep(torch.cat((vertices_source, vertices_target), dim=1))  # v->e
            h_e = torch.tanh(h_e)
            # compute e -> v message
            h_v = torch.zeros_like(enc_vertices)
            h_v = h_v.scatter_add(0, edges[:, 1].unsqueeze(1).repeat(1, enc_vertices.size(1)), h_e)

        h_v = self.f_v(h_v)
        h_v = torch.tanh(h_v)
        
        # Compute output predictions
        mu = self.f_mu(h_v)
        mu = torch.sigmoid(mu)
        
        return mu



In [8]:
class TrifocalLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, vertices_pred, vertices_gt):
        loss = (vertices_gt - vertices_pred).pow(2).mean()  # Changed from sum() to mean()
        return loss
    
# def cross_entropy_loss_func(edges_prob, edges_gt):
#     edges_gt_expanded = edges_gt.unsqueeze(-1).float()
#     loss_func = nn.BCEWithLogitsLoss()
#     loss = loss_func(edges_prob, edges_gt_expanded)
#     return loss

def edge_loss(edges_prob, edges_gt):
    # Expand edges_gt to match the shape of edges_prob
    print(edges_prob.shape)
    edges_gt_expanded = torch.zeros(edges_prob.shape, dtype=torch.float32)
    
    for i in range(edges_gt.shape[0]):
        u, v = edges_gt[i]
        print("u:", u)
        print("v:", v)
        print("edges_gt_expanded.shape:", edges_gt_expanded.shape)
        if i < edges_gt_expanded.shape[0]:
            if u < edges_gt_expanded.shape[1]:
                edges_gt_expanded[i, u] = 1
            if v < edges_gt_expanded.shape[1]:
                edges_gt_expanded[i, v] = 1
        print('new u', u)
        print('new v', v)
    # Compute the cross-entropy loss
    loss = -torch.sum(edges_gt_expanded.to(device) * torch.log(edges_prob + 1e-7))
    return loss



In [9]:
class KeypointPipeline(nn.Module):
    def __init__(self):
        super().__init__()
        self.keypoint_model = torch.load(weights_path).to(device)
        self.gnn_encoder = GNNEncoder()
        self.gnn_decoder = GNNDecoder()

    def forward(self, img):
#         print("image in keypoints eval phase", img.shape)
#         img = F.to_tensor(img).to(device)
        img.unsqueeze_(0)
        img = list(img)
        with torch.no_grad():
            self.keypoint_model.to(device)
            self.keypoint_model.eval()
            output = self.keypoint_model(img)
            
        img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()
        high_scores_idxs = np.where(scores > 0.7)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], \
            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)
        
        keypoints = []
        key_points = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append(list(map(int, kps[0,0:2])))
            key_points.append([list(map(int, kp[:2])) for kp in kps])

        labels = []
        for label in output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            labels.append(label)
            
        keypoints_ = [list(x) + [y] for (x, y) in sorted(zip(keypoints, labels))]
                
        keypoints = torch.stack([torch.tensor(kp) for kp in keypoints_]).float().to(device)
        enc_v, enc_e, edges = self.gnn_encoder(keypoints)
        
        vertices = self.gnn_decoder(enc_v, enc_e, edges)
        
        return vertices, enc_v, enc_e, edges



In [ ]:
# Define the model
model = KeypointPipeline()
model = model.to(device)

# Define the loss
criterion = TrifocalLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Load the trained model
model.keypoint_model = torch.load(weights_path).to(device)

num_epochs = 10  # Define your number of epochs

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=False)
dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=1, shuffle=True, collate_fn=collate_fn)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)


model.train()
# Epoch loop
for epoch in range(num_epochs):
    # For each batch in your training data
    for batch in data_loader_train:
        img_tuple, target_dict_tuple = batch
        img = img_tuple[0]
#         print(img.shape)
        target = target_dict_tuple[0]

        img = img.to(device)
        vertices_gt = target['keypoints'].to(device)
        num_vertices = vertices_gt.shape[0]
        print(num_vertices)
        vertices_gt[:, :, 2] = torch.arange(1, num_vertices+1).unsqueeze(1).to(device)

        # Forward pass
        vertices_pred, vertices_prob, edges_prob, edges_gt = model(img)
        
        # Compute the losses
        trifocal_loss = criterion(vertices_pred, vertices_gt)
        ce_loss = edge_loss(edges_prob, edges_gt)

        # Combined loss
        loss = trifocal_loss + ce_loss
#         loss = trifocal_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Copying files: 2662 files [00:00, 20779.49 files/s]
Copying files: 2662 files [00:00, 18517.59 files/s]
Copying files: 2662 files [00:00, 15029.40 files/s]


6
edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([7, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0'

6
edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0'

edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0')


edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0')


...

edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0')


6
edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0'

6
edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0'

6
edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([7, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0'

edges[:,0]:  tensor([0, 1, 2, 3, 4], device='cuda:0')
edges[:,1]:  tensor([1, 2, 3, 4, 5], device='cuda:0')
h1 that is problematic torch.Size([6, 128])
torch.Size([5, 128])
u: tensor(0, device='cuda:0')
v: tensor(1, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(0, device='cuda:0')
new u tensor(0, device='cuda:0')
u: tensor(1, device='cuda:0')
v: tensor(2, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(1, device='cuda:0')
new u tensor(1, device='cuda:0')
u: tensor(2, device='cuda:0')
v: tensor(3, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(2, device='cuda:0')
new u tensor(2, device='cuda:0')
u: tensor(3, device='cuda:0')
v: tensor(4, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(3, device='cuda:0')
new u tensor(3, device='cuda:0')
u: tensor(4, device='cuda:0')
v: tensor(5, device='cuda:0')
edges_gt_expanded.shape: torch.Size([5, 128])
new u tensor(4, device='cuda:0')
